## 1. INTRODUCTION

### 1.1. About

The purpose of this assigment is to explore and evaluate the capability of variaous machine learning algorithms in predicting which fish species you catch given different features such as gear used, geographical locations, boat size, and sea depth. We find it interesting to find out whether or not all types of gears are necessary. For instance, environmental consequences of bottom trawling; disturbance of seabed ecosystems.

Source: https://www.hi.no/hi/nyheter/2023/januar/traling-pavirker-livet-pa-havbunnen

A possible outcome of the assignment is to investigate if other techniques can get the same amount and species of fish. 



The dataset used in this assignment consits of a collection of fishing records, which among others include the gear, locations, boat sizes, and sea depths. Each record in the dataset also includes the species of fish caught during each trip. 

In our assignment, we will use Naïve Bayes and Random Forest to set a baseline for comparison. Furthermore, we will create a deep learning model, using neural networks, and experiment with an unsupervised learning model. The performance of the models will be evaluated based on their accuracy and precision in predicting the correct fish species. 

An important part of this assignment is feature selection and engineering, where we will attempt to identify which factors have the most significant influence on the prediction of fish species.



### 1.2. Import libraries

In [ ]:
import torch
from torch import nn 
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix
import pandas as pd 
import numpy as np 
import matplotlib as mlp
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans 
from matplotlib.colors import ListedColormap
%matplotlib inline 
import seaborn as sns
import os
from datetime import datetime 


# Unsupervised
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import umap.plot


# path that everyone can use 
workdir = os.getcwd()
parent = os.path.dirname(workdir)

file_path = 'elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv'
full_path = os.path.join(parent, file_path)
df = pd.read_csv(full_path, sep=";")  

_____________________________________________________

## 2. PRESENTATION OF DATA 
The data is distributed from Fiskedirektoratet from 2017-2019. 

### 2.1. Info

In [ ]:
df.info()

45 columns total. 

27 columns with 'object' values, which we later have to change. 

### 2.2. Head

In [ ]:
df.head()

### 2.3. Sample

In [ ]:
df.sample(n=5)

We compare the first five samples with five random ones. 'Bruttonnasje 1969' and 'Bruttotonnasje annen' have alternating NaN values. Some columns have commas; others have periods. 

________

## 3. INSPECTION OF DATA

### 3.1. Describe

In [ ]:
df.describe(include=object)

Object gives the count, unique entries in the columns, the most common value in each column, and the most common value's frequency. 

In [ ]:
df.describe(include=np.number)

- 'Havdybde start' and 'Havdybde stopp' contain positive values - counterintuitive as sea depth should be negative.
- 'Fangstår' max is 2018 - should be 2019. 
- 'Varighet' - large max value (125534)

### 3.2. Shape

In [ ]:
df.shape

df has 305,434 and 45 columns - useful for comparing after rows and columns are removed.




_________________________________________

## 4. PREPROCESSING DATA


### 4.1. Check for and remove duplicates

When reviewing the dataset, we noted potential duplicy and decided to remove duplicates to reduce redundancy, and increase accuracy and efficiency. 

In [ ]:
num_of_samples = df.shape[0]
deleted_samples = []

for sample_num in range(0,num_of_samples-1): 
    row = df.iloc[sample_num]
    next_row = df.iloc[sample_num + 1]
    # when the samples are identical they will be appended to the empty list
    append = True 
    # a loop for all samples that are unique
    for feature_values in zip(row, next_row): 
        if feature_values[0] != feature_values[1]: # if they are not identical, the next row is checked
            if pd.isna(feature_values[0]) and pd.isna(feature_values[1]):
                continue 
            append = False
            break 
    if append == True: 
        deleted_samples.append(sample_num)
 
df.drop(df.index[deleted_samples], inplace=True) # dropping all duplicate rows
print(deleted_samples)


There are nine identical rows. Given the small number of duplicates, it is highly unlikely that their removal will significantly impact the models performance.    

### 4.2. Remove columns


Dropping colums that will not be used simplifies the dataset, and focuses the analysis on important factors. Columns used in preprocessing will be dropped later. 


- We keep 'Startdato', and remove other columns associated with time. 
- We keep key columns like 'Havdybde start', 'Havdybde stopp', and positional coordinates, and remove redundant ones like 'Hovedområde' and 'Lokasjon'. 
- We keep 'Redskap FDIR', and remove other columns associated with gear. 
- We keep 'Hovedart FAO', and remove other columns associated with species. 
- We remove 'Fartøylengde' and 'Bredde' - and keep gross tonnage.

In [ ]:
df = df.drop(['Meldingstidspunkt', 'Melding ID', 'Meldingsdato', 'Meldingsklokkeslett', 'Starttidspunkt', 'Startklokkeslett', 'Stopptidspunkt', 'Stoppdato', 'Stoppklokkeslett', 'Varighet', 'Fangstår', 'Hovedområde start (kode)', 'Hovedområde stopp (kode)', 'Hovedområde stopp', 'Lokasjon stopp (kode)', 'Redskap FAO (kode)', 'Redskap FAO', 'Redskap FDIR (kode)', 'Hovedart FAO (kode)', 'Hovedart - FDIR (kode)', 'Art FAO', 'Art FAO (kode)', 'Art - FDIR (kode)', 'Art - FDIR', 'Art - gruppe (kode)', 'Art - gruppe', 'Rundvekt', 'Lengdegruppe (kode)', 'Lengdegruppe', 'Fartøylengde', 'Bredde'], axis=1)

### 4.3. NaN values


#### Summing NaN values

In [ ]:
df.isna().sum()

The amount of NaN values (except for Bruttotonnasje) are a few.  

#### 'Bruttotonnasje' NaN

We chose 'Bruttotonnasje' to measure vessels because it indicates internal volume and storage capacity, while 'Bredde' and 'Fartøylengde' only measure surface area.

'Brutotonnasje 1969' and 'Bruttotonnasje annen' have about 300,000 NaN values combined. To preserve data, we replace these with 0. 

In [ ]:
df['Bruttotonnasje 1969'] = df['Bruttotonnasje 1969'].fillna(0)
df['Bruttotonnasje annen'] = df['Bruttotonnasje annen'].fillna(0)

#### Dropping NaN values

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.describe(include=np.number)

df has been reduced to 13 columns and 10,000 fewer samples, which is a reasonable redution for the size of our dataset. 

### 4.4. Converting columns to numerical format


Useful because we will later use these columns in mathematical operations. 

In [ ]:
features_convert = ['Bruttotonnasje 1969', 'Bruttotonnasje annen', 'Havdybde start', 'Havdybde stopp', 'Startposisjon bredde', 'Startposisjon lengde', 'Stopposisjon lengde', 'Stopposisjon bredde']

for kolonne in features_convert: 
    df[kolonne] = df[kolonne].astype(str) # convert columns into string
    df[kolonne] = df[kolonne].str.replace(',', '.').astype(float) #replace comma with period, and convert back to float

_____________________________________________________

## 5. INSPECTION AND VISUALIZATION OF SELECTED FEATUERS 

### 5.1. 'Hovedart FAO'

In [ ]:
species_counts = df['Hovedart FAO'].value_counts(sort=True) # sorting in descending order 
for species, count in species_counts.items():
    print(f"'{species}': {count}")

64 unique species. Several are caught only a few times. 

In [ ]:
# making a list of the top 20 species
top_20 = species_counts[:20]
top_species = top_20.index.tolist()
top_species

In [ ]:
rest_of_species = species_counts[20:] 
species_to_remove = rest_of_species.index.tolist()

df = df.loc[df['Hovedart FAO'].isin(species_to_remove) == False] # removing all species that are not in top 20 

### 5.2. 'Havdybde'

In [ ]:
print((df['Havdybde start'] > 0).sum())
print((df['Havdybde stopp'] > 0).sum())


1053 values in 'Havdybde start' and 1185 in 'Havdybde stopp' are positive. 

These numbers are small, but also irregularities that have to be addressed later. We want to ensure accuracy of our data, and avoid skewing the model with inaccurate information.   

A visual summary of sea depth will help understanding the range. 

Plotting 'Havdybde start' and 'Havdybde stopp' together because we want a quick summary. 

In [ ]:
# Beregn antall forekomster av hver unik verdi
start_counts = df['Havdybde start'].value_counts().sort_index() 
stopp_counts = df['Havdybde stopp'].value_counts().sort_index()

plt.plot(start_counts, label='Havdybde start') # Plotting frequency 
plt.plot(stopp_counts, label='Havdybde stopp')

plt.xlabel("Havdybde verdi")
plt.ylabel("Antall forekomster")
plt.title("Frekvens av 'Havdybde start' og 'Havdybde stopp'")
plt.legend()

plt.show()

Key observations:

- Several sea depths occur few times, suggesting specialized or uncommon fishing zones. 
- Confirming positive values.

https://pandas.pydata.org/docs/getting_started/intro_tutorials/04_plotting.html

Converting all positive sea depths into negative values maintains integrity of the dataset, and ensures accurate training.

In [ ]:
havdybde_start_max = df['Havdybde start'].max()
havdybde_stopp_max = df['Havdybde stopp'].max()

In [ ]:
df.loc[(df['Havdybde start'] >= 0) & (df['Havdybde start'] <= havdybde_start_max), 'Havdybde start'] = -df['Havdybde start']
df.loc[(df['Havdybde stopp'] >= 0) & (df['Havdybde stopp'] <= havdybde_stopp_max), 'Havdybde stopp'] = -df['Havdybde stopp']

Constructing a new feature calculated as the mean sea depths, will simplify the dataset. 

In [ ]:
df['Havdybde avg'] = df[['Havdybde start', 'Havdybde stopp']].mean(axis=1)

In [ ]:
df['Havdybde avg'].describe()

Key observations: 

- 25% of the data falls below -274 meters.
- mean depths is -225.77 meters.
- std is 195 meters, suggesting a considerable variation in depths. 

Using std and mean depth (-225.77+195, -225.77+195) to get a number to start with. 

In [ ]:
total_samples = df['Hovedart FAO'].count()
samples_in_range = df[(df['Havdybde avg'] >= -450) & (df['Havdybde avg'] <= -30)].count()
percentage_in_range = (samples_in_range['Hovedart FAO'] / total_samples) * 100
print(percentage_in_range)

95% of the fishing occurs between -30 and -450 meters, reasonable amount to keep. 

Plotting 'Havdybde avg' so we can evaluate 'Havdybde avg'. 

In [ ]:
start_counts = df['Havdybde avg'].value_counts().sort_index() # Calculates the number of occurrences of each unique value.

plt.plot(start_counts, label='Havdybde avg') # Plotting frequencies 
plt.xlabel("Havdybde verdi")
plt.ylabel("Antall forekomster")
plt.title("Frekvens av 'Havdybde avg'")
plt.legend()
plt.show()

Key observations: 

- Positive values have been eliminated. 
- Still a wide range of depths. 

We find it fitting to keep sea depths between -30 and -450 meters, as this contains the most relevant information for our task, and will ensure more precision and reliability.


In [ ]:
df["Havdybde avg"]= df["Havdybde avg"].clip(upper = -30) 
df = df[df['Havdybde avg'] > -450]
df['Havdybde avg'].describe()

Ensuring that the top species are within the specified depth range.

In [ ]:
havdybde_df = df[['Havdybde start', 'Havdybde stopp', 'Hovedart FAO']].copy()

havdybde_filtered = havdybde_df[havdybde_df['Hovedart FAO'].isin(top_species)]

In [ ]:
depth_ranges = havdybde_filtered.groupby('Hovedart FAO')['Havdybde start'].agg(['min', 'max']).reset_index() # Find the depth ranges for each top species 

print(depth_ranges)

In [ ]:
start_counts = df['Havdybde avg'].value_counts().sort_index()  # Calculates the number of occurrences of each unique value.

plt.plot(start_counts, label='Havdybde avg') # Plotting frequencies 
plt.xlabel("Havdybde avg verdi")
plt.ylabel("Antall forekomster")
plt.title("Frekvens av 'Havdybde avg'")
plt.legend()
plt.show()

The plot shows us that the values are distributed more evenly. 

### 5.3. 'Redskap FDIR'

Gear is an important feature to inspect. 

Different gear types are tailored to specific species by habitat and size, with some optimized for bulk catching and others for selective targeting. 

In [ ]:
all_equipment = []
columns = list(df['Redskap FDIR'])
for equipment in columns: 
    if equipment not in all_equipment: 
        all_equipment.append(equipment) #appends all unique gear types to empty list

for item in all_equipment:
    print(item)

print(len(all_equipment))

There are 18 unique gear types (including one NaN). This helps create a basis for sorting gear, and find out what gear typically catches different species. 

These groups are sorted based on where they generally fish, and each group catches the same species. 

https://www.fiskeridir.no/Yrkesfiske/Tema/redskapshefte/Redskapshefte.pdf

Four gears are removed because these are used less than 500 times, to make it easier for the models later. 

In [ ]:
redskap_top = ['Flytetrål', 'Dobbeltrål', 'Snurpenot/ringnot', 'Flytetrål par']
redskap_bottom = ['Bunntrål', 'Snurrevad', 'Teiner', 'Bunntrål par', 'Reketrål']
redskap_settegarn = ['Settegarn']
redskap_udef_garn = ['Udefinert garn', 'Andre liner']
redskap_udef_trål = ['Udefinert trål']

vessel_group will be applied to gear in the feature engineering function later.

In [ ]:
# function to categorize gear into one of five groups
def vessel_group(equipment, group): 
     # if gear is present in any of the predefined lists, and found in corresponding group list, return 1
    if group == 1 and equipment in redskap_top: 
         return 1 
    elif group == 2 and equipment in redskap_bottom: 
         return 1 
    elif group == 3 and equipment in redskap_settegarn: 
         return 1 
    elif group == 4 and equipment in redskap_udef_garn: 
         return 1 
    elif group == 5 and equipment in redskap_udef_trål: 
         return 1 
    else: 
        return 0 

Grouping the gear may lead to inconsistencies and biases. Each gear may catch more than one species, and if each gear is typically used in a particular area, it might affect which species we expect to catch. 

## 5.3. 'Startdato' 


In [ ]:
df['Startdato'] = pd.to_datetime(df['Startdato'], format='%d.%m.%Y') # parse data to get date format 

In [ ]:
df['Month'] = df['Startdato'].dt.month # extract month 
df['Month'].describe()

In [ ]:
# function that determines the season of given dates based on month and day
def get_season(date: datetime, north_hemisphere: bool = True) -> str: 
    now = (date.month, date.day)
    if (3, 21) <= now < (6, 21):
        season = 'Spring' if north_hemisphere else 'Fall'
    elif (6, 21) <= now < (9, 21):
        season = 'Summer' if north_hemisphere else 'Winter'
    elif (9, 21) <= now < (12, 21):
        season = 'Fall' if north_hemisphere else 'Spring'
    else:
        season = 'Winter' if north_hemisphere else 'Summer'

    return season

df['Season'] = df['Startdato'].apply(get_season)

https://stackoverflow.com/questions/16139306/determine-season-given-timestamp-in-python-using-datetime

In [ ]:
season_dummies = pd.get_dummies(df['Season'])
df = pd.concat([df, season_dummies], axis=1) # concatenates dummies as columns in df

### 5.4. Trekkavstand

In [ ]:
df['Trekkavstand'].describe()

Large difference between max value and top 75%, suggesting outliers. 

In [ ]:
trekk_counts = df['Trekkavstand'].value_counts()
print(trekk_counts)

28948 unique values in 'Trekkavstand. 

In [ ]:
# Seaborn histogram
sns.set(style='whitegrid')
sns.histplot(df['Trekkavstand'], bins=50, kde=False)  # kde=False removes estimated density curve
plt.title('Histogram av Trekkavstand')
plt.xlabel('Trekkavstand')
plt.ylabel('Frekvens')
plt.show()

It looks like most values are concentrated in the lower end of the scale, which might indicate errors in the data, or that towing distance is sually short (with a small number of outliers in other bins, as we can't see these). 

Checking for outliers in 'Trekkavstand', by checking towing distance for each top species, and exploring if 'Trekkavstand' affects species. 

In [ ]:
df_filtered = df[df['Hovedart FAO'].isin(top_species)]

plt.figure(figsize=(10, 6))
for species in top_species:
    species_data = df_filtered[df_filtered['Hovedart FAO'] == species]
    plt.scatter(species_data['Trekkavstand'], [species] * len(species_data), alpha=0.5, label=species)
plt.xlabel('Trekkavstand')
plt.ylabel('Hovedart FAO')
plt.title('Scatterplot av Trekkavstand for de 20 mest fiskede artene')
plt.legend()
plt.show()

The plot confirms the expected outliers. 

In [ ]:
trekkavstand_sort = df['Trekkavstand'].sort_values(ascending=True)
len(trekkavstand_sort)
print(trekkavstand_sort)

In [ ]:
trekk_list = trekkavstand_sort.tolist()


In [ ]:
upper_trekk_value = trekk_list[int(len(trekkavstand_sort)*0.95)] 
upper_trekk_value

95% of the values in 'Trekkavstand' are less than or equal to 42146, which again indicates outliers and allows us to set a threshold below. 

In [ ]:
df['Trekkavstand'] = df['Trekkavstand'].clip(upper=upper_trekk_value)


In [ ]:
value_counts = df['Trekkavstand'].value_counts()
zero_gear_rows = df[df['Trekkavstand'] == 0]
unique_gear_types = zero_gear_rows['Redskap FDIR'].unique() # list of unique values in 'Redskap FDIR'

print(unique_gear_types)

In [ ]:
gear_with_zero_trekkavstand = df[df['Trekkavstand'] == 0]['Redskap FDIR'].value_counts()

print(gear_with_zero_trekkavstand)

Notes: 

Over 10,000 of the gears with towing distances of 0 are unspecified gear types. We assume that these might be errors, and therefore opted to modify these. 

In [ ]:

for gear in unique_gear_types: 
    # calculate mean (excluding 0) 
    mean_value = df[(df['Redskap FDIR'] == gear) & (df['Trekkavstand'] != 0)]['Trekkavstand'].mean()
    if pd.notna(mean_value):
        # replace 0 with mean value 
        df.loc[(df['Redskap FDIR'] == gear) & (df['Trekkavstand'] == 0), 'Trekkavstand'] = mean_value


In [ ]:
df['Trekkavstand'].describe()

In [ ]:
sns.set(style='whitegrid')

# Lager et histogram av Trekkavstand med Seaborn
sns.histplot(df['Trekkavstand'], bins=50, kde=False)  # kde=False fjerner estimert tetthetskurve
plt.title('Histogram av Trekkavstand')
plt.xlabel('Trekkavstand')
plt.ylabel('Frekvens')
plt.show()

The histogram shows a more evenly distribution of the bins. 
With more time we would look more into the bin with the highest numbers and what we could do with this. 

In [ ]:
df_filtered = df[df['Hovedart FAO'].isin(top_species)]

plt.figure(figsize=(10, 6))
for species in top_species:
    species_data = df_filtered[df_filtered['Hovedart FAO'] == species]
    plt.scatter(species_data['Trekkavstand'], [species] * len(species_data), alpha=0.5, label=species)
plt.xlabel('Trekkavstand')
plt.ylabel('Hovedart FAO')
plt.title('Scatterplot av Trekkavstand for de 20 mest fiskede artene')
plt.legend()
plt.show()


The scatterplot shows a more distict pattern, however there are still too many samples for this kind of plotting. For instance, it looks like Kolmule typically has a long towing distance. It is not possible to make assumptions about themore frequently caught species. 

In [ ]:
df_filtered = df[df['Hovedart FAO'].isin(top_species)]

plt.figure(figsize=(10, 6))
for species in top_species:
    species_data = df_filtered[df_filtered['Hovedart FAO'] == species]['Trekkavstand']
    median = species_data.median()
    std = species_data.std()
    typical_range = species_data[(species_data >= median-std) & (species_data <= median+std)]
    
    # Plot a line for the median value
    plt.hlines(y=species, xmin=median-std, xmax=median+std, color='grey', alpha=0.5)
    
    # Scatter plot for points within one standard deviation of the median
    plt.scatter(typical_range, [species] * len(typical_range), alpha=0.5, label=species)

plt.xlabel('Trekkavstand')
plt.ylabel('Hovedart FAO')
plt.title('Scatterplot av Trekkavstand for de 20 mest fiskede artene')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  # Move the legend out of the plot
plt.tight_layout()  # Adjust layout to accommodate the legend
plt.show()

This scatterplot makes it much easier to say something about the towing distance for all species. 
- The assumtion about Kolmule was correct
- The largest fish groups Torsk, Sei, and Hyse all mainly have a short towing distances. 

### 5.5. Position

In [ ]:
df[['Startposisjon bredde', 'Startposisjon lengde', 'Stopposisjon bredde', 'Stopposisjon lengde']].describe()

'Startposisjon bredde' and 'Startposisjon lengde' have very similar values. Same goes for length. 
We will therefor later drop both stop positions, and keep both start positions, to reduce features. 

##### Average latitude and longitude
Gives a more nuanced input for the models without sacrificing important information. Reduces some noise. In addition, models that are trained on generalized features, tend to generalize better on new data. May also reduce the training time for the models. 


In [ ]:
# groups all the unique values from 'Hovedområde start' togethe
grouped_start_bredde = df.groupby('Hovedområde start')['Startposisjon bredde']
grouped_start_lengde = df.groupby('Hovedområde start')['Startposisjon lengde']
grouped_stopp_bredde = df.groupby('Hovedområde start')['Stopposisjon bredde']
grouped_stopp_lengde = df.groupby('Hovedområde start')['Stopposisjon lengde']

# calculate sum of latitude and longtude of each of the unique main areas. 
grouped_lengde_count = grouped_start_lengde.count() + grouped_stopp_lengde.count()
grouped_lengde_sum = grouped_start_lengde.sum() + grouped_stopp_lengde.sum()
grouped_bredde_count = grouped_start_bredde.count() + grouped_stopp_bredde.count()
grouped_bredde_sum = grouped_start_bredde.sum() + grouped_stopp_bredde.sum()
# calculates the average latitude and logitude. 
average_lengde = grouped_lengde_sum/grouped_lengde_count
average_bredde = grouped_bredde_sum/grouped_bredde_count

print(average_bredde)

Confirms that each unique area in 'Hovedområde FAO' has only one, average coordinate assigned to it.  

##### Clustering average latitude and longitude 

We convert average_bredde and average_lengde into numpy arrays, and then stack them vertically (vstack), and transposing them to get the data format we need for clustering.

In [ ]:
arr1 = np.array(average_bredde.values)
arr2 = np.array(average_lengde.values)
# transpose flips the horisontal and vertical values of the matrix 
out_arr = np.vstack((arr2, arr1)).transpose()
print(out_arr.shape)

(46, 2): 46 unique areas ('Hovedområde start'), each with two features ('average_bredde', 'average_lengde'). 


As there are 46 unique areas, we wish to cluster these based on proximity. Therefor we use KMeans clustering. 
- 11 clusters: we tested with more/fewer clusters, however, we concluded the clusters to be too specific or broad then.
- Random_state: 0 for same output each time.

Source: https://scikit-learn.org/stable/modules/generated/sklearn.cluster.k_means.html

In [ ]:
km = KMeans(n_clusters= 11, random_state=0)
# fitting model 
km.fit(out_arr)
# cluster labels are retrieved for each data point in out_arr 
clustered_groups = km.labels_
print(clustered_groups)
print(len(clustered_groups))

47 unique areas now has a cluster label.

In [ ]:
customcmap = ListedColormap(['crimson', 'mediumblue', 'darkmagenta', 'lightgreen', 'yellow', 'pink', 'indigo', 'brown', 'black', 'lightblue', 'orange'])
fig, ax = plt.subplots(figsize =(8, 8))
# Each point's size is set to 100, the colour corresponds to the cluster label, and the color is chosen from the 'custommap' variable
plt.scatter(x = average_bredde, y = average_lengde, s = 100, c = clustered_groups, cmap = customcmap)
ax.set_xlabel(r'average_bredde', fontsize = 14)
ax.set_ylabel(r'average_lengde', fontsize = 14)
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
plt.show

Points that have the same colour are in the same cluster, and these are in quite close proximity to each other. 

Source: https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.ListedColormap.html

##### Encoding Location Data 

Below we are going to place each sample from the dataset into one of the 11 clusters made above, and one hot encode the clusters. 
Transofrming categorical data into binary format is more suitable for the models

In [ ]:
def location(main_area, groupnum_label): # group numbers 
     # Finds the index of the given area in the list 'average_bredde.index' 
     index_område = list(average_bredde.index).index(main_area)
     group_no = groupnum_label[index_område]
     # returns the correspondig label from the list of labels.
     return group_no

def encoded_location(cluster_code, group_location): # one hot encode clusters 
     # compares a clustere code (0 to 10) with the clustered group labels (also 0 to 10)
     if cluster_code == group_location:  # If cluster code = clustered_groups label match the sample belongs to this cluster

          return 1 
     else: 
          return 0

_______________________________________________________________________________

## 6. FEATURE ENGINEERING FUNCTION

The code applies a series of alterations to the DataFrame. We chose to compile the implementation of all new features in one function, as it makes the code neater. 

Merging the two 'Bruttotonnasje' features into one single feature simplifies analysis by reducing complexity and managing correlated variables more effectively.

In [ ]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy() # copy the DataFrame to avoid modifying the original dataset directly. 
    df['Bruttotonnasje'] = np.maximum(df['Bruttotonnasje 1969'], df['Bruttotonnasje annen']) # Merging 'Bruttotonnasje 1969' and 'Bruttotonnasje annen' into a single feature
    df['Temp_location'] = df['Hovedområde start'].apply(location, groupnum_label = clustered_groups) # temporary location for one hot encoding 
    for clustered_group_no in range(0, 11): # loop through group labels 
        group_location = 'group_location' + str(clustered_group_no)
        df[group_location] = df['Temp_location'].apply(encoded_location, group_location=clustered_group_no) # apply hot_location to one-hot encode the locations
    # create 5 new features using vessel_group
    df['Redskap top'] = df['Redskap FDIR'].apply(vessel_group, group=1)
    df['Redskap bottom'] = df['Redskap FDIR'].apply(vessel_group, group=2)
    df['Redskap settegarn'] = df['Redskap FDIR'].apply(vessel_group, group=3)
    df['Redskap udefinert garn'] = df['Redskap FDIR'].apply(vessel_group, group=4)
    df['Redksap udefinert trål'] = df['Redskap FDIR'].apply(vessel_group, group=5)

    return df 

In [ ]:
df = feature_engineering(df)
df = df.drop(['Havdybde start', 'Havdybde stopp', 'Season', 'Startdato', 'Temp_location', 'Hovedområde start', 'Lokasjon start (kode)', 'Stopposisjon bredde', 'Stopposisjon lengde', 'Redskap FDIR', 'Bruttotonnasje 1969', 'Bruttotonnasje annen'], axis = 1)

In [ ]:
df.head() # confirming feature engineering

_______________________________________________________________

## 7. LABEL ENGINEERING 

Since our models are going to predict fish species, 'Hovedart FAO' is our label. 

### 7.1. Preprocessing 'Hovedart FAO' 

One-hot encoding label into binary labels. 

In [ ]:
def fish_group(df_fish, top_fish): 
    if df_fish == top_fish: 
        return 1
    else: 
        return 0

train_labels = pd.DataFrame() # create a new dataframe that will be filled up with all the species within the top_fish

# loop through each fish species in top_species to fill up trainlabels with relevant samples, and apply fish_group to onehotencode. 
for fish in top_species: 
    train_labels[fish] = df['Hovedart FAO'].apply(fish_group, top_fish=fish)

print(df['Hovedart FAO'].value_counts().shape) 



### 7.2. Unsupervised model label 

Creating labels for use in the unsupervised model before dropping 'Hovedart FAO'. feature.

In [ ]:
train_labels.sample(10)

In [ ]:
rest_of_species_unsup = species_counts[12:]
species_to_remove_unsup = rest_of_species_unsup.index.tolist()

In [ ]:
top_12 = species_counts[:12]
top_12_species = top_12.index.tolist()
top_12_species

In [ ]:
unsupervised_df = df.copy()
unsupervised_df = unsupervised_df.loc[df['Hovedart FAO'].isin(species_to_remove_unsup) == False] # Removing fish species not equal to the 12 top species
train_labels_unsupervised = unsupervised_df['Hovedart FAO'] # creating a label based on the 12 fish species 

In [ ]:
df = df.drop('Hovedart FAO', axis=1)

In [ ]:
train_labels.shape

In [ ]:
df.describe()

In [ ]:
df.shape

____________________

## 8. NORMALIZATION 

Source: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
scaler = StandardScaler()

feature_to_scale = df[['Month', 'Bruttotonnasje', 'Havdybde avg', 'Trekkavstand', 'Startposisjon bredde', 'Startposisjon lengde']]
feature_scaled = scaler.fit_transform(feature_to_scale) # using StandardScaler to scale features 
df[['Month', 'Bruttotonnasje', 'Havdybde  avg', 'Trekkavstand', 'Startposisjon bredde', 'Startposisjon lengde']] = feature_scaled

In [ ]:
df.describe() 

Confirms successful scaling. Values look reasonably distributed within the columns.

Convert DataFrame into PyTorch tensors.  

In [ ]:
x_data = torch.tensor(df.values.astype(np.float32))
y_data = torch.tensor(train_labels.values, dtype=torch.float32)

type(x_data), type(y_data) # type() to confirm

_____

## 9. Splitting data into training and validation sets
Tensordataset: wraps tensors into a dataset for indexed access to paired data.
Dataloader: uses dataset and loads data in batches. Beneficial for large datasets. 

https://www.appsilon.com/post/pytorch-neural-network-tutorial 

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size = 0.25, shuffle=True)

train_data = TensorDataset(x_train, y_train)
validation_data = TensorDataset(x_valid, y_valid)

batch_size = 5000
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
validation_loader = DataLoader(validation_data, batch_size=len(validation_data.tensors[0])) 

print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_valid:', x_valid.shape)
print('y_valid:', y_valid.shape)

# SUPERVISED MODELS

## 1. RANDOM FOREST 

We chose this model because it is generally effective for complex datasets. Given our features' non-linear relationships, it'll hopefully fit our task. 
Source: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
# Random Forest Classifier 
rfc = RandomForestClassifier()

# Fit the model to the training data
rfc.fit(x_train.numpy(), y_train.numpy().argmax(axis=1))

# Make predictions on the validation set
y_pred = rfc.predict(x_valid.numpy())

# Calculate the overall accuracy
overall_accuracy = accuracy_score(y_valid.numpy().argmax(axis=1), y_pred)
print(f'Overall Accuracy: {overall_accuracy}')

# Print the classification report
report = classification_report(y_valid.numpy().argmax(axis=1), y_pred, target_names=top_species, zero_division=1)
print(report)


The accuracy is quite high likely due its ability to capture complex relationships. 

## 2. NAIVE BAYES 

We chose Naive Bayes due to its simplicity and efficiency. 

Source: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html

In [ ]:

# Gaussian Naive Bayes classifier
gnb = GaussianNB()

# Convert y_train and y_valid to class labels
y_train_labels = np.argmax(y_train.numpy(), axis=1)
y_valid_labels = np.argmax(y_valid.numpy(), axis=1)

# Fit the model on the training data
gnb.fit(x_train.numpy(), y_train_labels)

# Make predictions on the validation data
y_pred = gnb.predict(x_valid.numpy())

# Calculate the overall accuracy
overall_accuracy = accuracy_score(y_valid_labels, y_pred)
print(f'Overall Accuracy: {overall_accuracy}')

# Print the classification report
report = classification_report(y_valid_labels, y_pred, target_names=top_species, zero_division=1)
print(report)

The model's accuracy is quite low. Upon investigating, this isn't surprising. The model's simplicity seemingly hasn't captured complex patterns present in the dataset, and might perform poorly when the data distribution deviates from Gaussian assumption. These are some weaknesses with Gaussian Naive Bayes.

## 3. DEEP LEARNING 

### 3.1. Neural Network Class 

FishSpecies defines a neural network for classifying the species. 
__init__ sets up layers, Relu and dropout. 
forward function passes through the layers. 

Source: https://www.nickersonj.com/posts/pytorch-tabular/, https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/


ReLU activation function transforms negative numbers to 0, and keeps positive numbers unchanged. 

Source: https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/


Dropout is used to prevent overfitting. 15% of the nodes in the neural network are removed temporarily during the training of the model. 

Source: https://machinelearningmastery.com/

In [ ]:
class FishSpecies(nn.Module): 
    def __init__(self):
        super().__init__()
        # Creating 5 neuron layers 
        self.linear1 = nn.Linear(27, 64) 
        self.linear2 = nn.Linear(64, 128)
        self.linear3 = nn.Linear(128, 96)
        self.linear4 = nn.Linear(96, 32)
        self.linear5 = nn.Linear(32, 20)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.15)


    def forward(self, x):
        z = self.linear1(x) # 21 features as input going to 64 nodes in the next layer. 
        z = self.relu(z)
        z = self.dropout(z)
        z = self.linear2(z)
        z = self.relu(z)
        z = self.dropout(z)
        z = self.linear3(z)
        z = self.relu(z)
        z = self.dropout(z)
        z = self.linear4(z)
        z = self.relu(z)
        z = self.linear5(z)
    
        return z # Returns one value per class (from the label)


In [ ]:
model = FishSpecies()
model

### 3.2 Learning rate, Loss function, and Optimizer

Cross-entropy and ADAM are commonly used in multiclass classification. 
Well-suited for dealing with probabilities that sum to 1 (softmax)

Sources: https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1, 
https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/


After trying out different values, we decided to set the learning rate to 0.002, as it balances learning speed and risk of overshooting the loss function's minimum. We chose Cross-entropy for its suitability for mulit-class problems, and used the ADAM optimizer for its adaptive learning rate, enhancing performance with our large dataset. 


Source: https://machinelearningmastery.com/learning-rate-for-deep-learning-neural-networks/


In [ ]:
learning_rate = 0.002

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

Testing package before training  
Used to look at the logits (raw data) before and after Softmax, comparing with labels from y_train, and checking Argmax function. 

In [ ]:
logits = model(x_train)
print('logits:', logits[:5]) 
loss = loss_fn(logits, y_train)

softmax = nn.Softmax(dim=1)
optim = softmax(logits[:1])
print(optim)
print(torch.sum(optim,dim=0))

y_pred = torch.argmax(logits[:5], dim=1)
print(y_pred)

Accuracy Function 



In [ ]:
def accuracy_fn(y_true, y_pred): 
    acc = accuracy_score(y_true, y_pred) * 100
    return acc

Source: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

### 3.4. Training 

Source: https://www.nickersonj.com/posts/pytorch-tabular/

Source batch, epoch: https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/

Source tqdm: https://adamoudad.github.io/posts/progress_bar_with_tqdm/

In [ ]:
epochs = 100
epoch_count, train_loss_values, valid_loss_values = [], [], [] # keeps tracks of epoch number, training loss values and validation loss values for the output plotting


for epoch in range(epochs): # trains model by repeatedly going through the data, while updating the model to improve prediction
    # Keeping track of loss and accuracy values of all batches in all epochs 
    epoch_loss = []
    epoch_acc = [] 
    model.train() # Puts the model in training mode 

    with tqdm(train_loader, unit="batch") as bar: # plotting bars
        bar.set_description(f"Epoch {epoch}")

        for x_batch, y_batch in bar: 
            y_logits_batch = model(x_batch) # forward pass to get the predictions the model thinks is correct based on a forward pass through all samples. 
            loss = loss_fn(y_logits_batch, y_batch) # compute the loss 
            
            optimizer.zero_grad() # resets the gradients so they don't accumulate each iteration
            loss.backward() # backpropagates the prediction loss
            optimizer.step() # gradient descent: updates the weights  
           
            y_pred = torch.argmax(y_logits_batch, dim=1) # 
            y_train_indices = torch.argmax(y_batch, dim=1)
            acc = accuracy_fn(y_train_indices, y_pred) # calculate accuracy 
            epoch_loss.append(float(loss))
            epoch_acc.append(float(acc))
            bar.set_postfix(
                loss=float(loss),
                acc=float(acc)
            )

    # evaluation mode for the validation data
    model.eval() 
     
    with torch.inference_mode(): # ensures that model parameters are not updated (resets for validation) and disables gradient calculations and dropout. 

        valid_logits = model(x_valid).squeeze() 
        valid_pred = torch.argmax(valid_logits, dim=1)    
        valid_loss = loss_fn(valid_logits, y_valid)
        y_valid_indices = torch.argmax(y_valid, dim=1)
        valid_acc = accuracy_fn(y_valid_indices, valid_pred) 

    # for printing progress 
    if epoch % int(epochs / 20) == 0:
        print(f'Epoch: {epoch:4.0f} | Train Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Validation Loss: {valid_loss:.5f}, Accuracy: {valid_acc:.2f}%')
        epoch_count.append(epoch)
        train_loss_values.append(loss.detach().numpy())
        valid_loss_values.append(valid_loss.detach().numpy())


Througout the epochs the validation loss has a lower value than the train loss. This might indicate that the model is generalizing instead of memorizing the training data. The Validation Accuracy is higher than the training accuracy, which is an indicator that the model has good generalization and does not overtrain.



In [ ]:
overall_accuracy = accuracy_score(y_train_indices, y_pred)
print(f'Overall Accuracy: {overall_accuracy}')

cr = classification_report(y_train_indices, y_pred, target_names=top_species)
print(cr)

Precision measures accurcay of positive predictions. Dvptvannsreke, Sild and Snøkrabbe are species with high scores. I.e. Lysing and Kolmule has a lower value indicating that the model does not find as good patterns here. 

## 4. Conclusion Supervised

To conclude we feel happy with how far we got. The feature engineering could have been done very differently (i.e., cropping differently, more/less groups for gear and location, different features).The features we tested with but decided to not to keep are worth mentioning. For instance, we cropped 'Varighet' to 440min, and tried with day and year (difficult to say what works best as the dataset primarily consists of dates in 2018.), but it didn't affect the accuracy.  
More in-depth plotting could also have made the analysis more thorough and helped the feature engineering. 

When it comes to the supervised methods we quickly saw that the Random Forest had a higher accuracy than Naïve Bayes. This was likely due to its ability to capture complex relationships and effectively handle the features in our dataset. We should have used multiple decision trees with optimal parameters from grid search to improve the accuracy. Naïve Bayes received a quite low accuracy, but this was as expected, due to the size of the dataset. The model might have been negatively impacted when the data distribution didn't align with the Gaussian assumption. With the deep learning method we would have liked to test further with neuronlayers, batchsizes, and epochs, and other activation functions and dropouts, but we feel that the feature engineering was somewhat suitable for the model.


# UNSUPERVISED MODEL 

The structure and some of the code is inspired by: https://www.kaggle.com/code/samuelcortinhas/intro-to-pca-t-sne-umap. 

Will therefore use PCA, t-SNE and UMAP. 

In [ ]:
df.info()

Checking for columns to keep for the unsupervised model. Will remove all one hot encoded features. 

We create a dictionary of the original species with their enlabeled numbers. 
This will later be used to analyze the clusters to see if there are any patterns regarding the fish species. 

In [ ]:
num_unsupervised = 20000 # specifying the amount of samples to use 

# Labelencoding the fish species to numbers
le = preprocessing.LabelEncoder() 
train_labels_unsupervised = le.fit_transform(train_labels_unsupervised[:num_unsupervised])

# creating a dict that maps the numbers from label encoding to their corresponding fish species
le_name_mapping = dict(zip(le.transform(le.classes_), le.classes_))
le_name_mapping 


Inspired by this source: 
https://stackoverflow.com/questions/42196589/any-way-to-get-mappings-of-a-label-encoder-in-python-pandas

In [ ]:
# Converting the dictionary to a list 
label_fishes = list(le_name_mapping.values())
label_fishes

Updating df to chosen features.

In [ ]:
unsupervised_df = unsupervised_df[['Startposisjon bredde', 'Startposisjon lengde', 'Trekkavstand', 'Month', 'Havdybde avg', 'Bruttotonnasje']]

Scaling the features before clustering 

In [ ]:
feature_to_scale = unsupervised_df[['Startposisjon bredde', 'Startposisjon lengde', 'Trekkavstand', 'Month', 'Havdybde avg', 'Bruttotonnasje']]
feature_scaled = scaler.fit_transform(feature_to_scale)
unsupervised_df[['Startposisjon bredde', 'Startposisjon lengde', 'Trekkavstand', 'Month', 'Havdybde avg', 'Bruttotonnasje']] = feature_scaled

In [ ]:
# Confirming that the data is scaled  
unsupervised_df.describe()

In [ ]:
# Converting numpy array values to torch tensors  
x_data_unsupervised = torch.tensor(unsupervised_df.values.astype(np.float32))

#### Pie Chart all fish 
Visualize our 20 fish species for later comparison.


https://www.geeksforgeeks.org/plot-a-pie-chart-in-python-using-matplotlib/

In [ ]:
# Count the occurrences of each label
label_counts = np.bincount(train_labels_unsupervised[:num_unsupervised])

percentages = label_counts / label_counts.sum()

threshold = 0.02

# Filter species names based on the threshold
species_names = [le_name_mapping[i] for i in range(len(label_counts)) if percentages[i] >= threshold]
filtered_label_counts = [count for count in label_counts if count / label_counts.sum() >= threshold]

# Now plot the pie chart with filtered labels and counts
plt.figure()
plt.pie(filtered_label_counts, labels=species_names, autopct='%1.1f%%', startangle=90)
plt.title("Distribution of Fish Species")
plt.show()


Notes: Torsk, Dypvannsreke, Sei and Hyse are the only species over 2%. 

## 1. PCA 

In [ ]:
# Initialize PCA, 2 dimentions 
pca = PCA(n_components=2) 

# Transform xdata to dataframe using pca and reduce to two dimensions
X_pca = pca.fit_transform(x_data_unsupervised[:num_unsupervised])

# new dataframe using pca values in two columns
principal_df = pd.DataFrame(data = X_pca, columns = ['PC1', 'PC2']) 

# Shape and preview
print(principal_df.shape)
principal_df.head()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(principal_df.iloc[:,0], principal_df.iloc[:,1], s=40)
plt.title('PCA plot in 2D')
plt.xlabel('PC1')
plt.ylabel('PC2')

Adding colors to each point, one color for each fish.

In [ ]:
# def spec generates 12 different colors
def spec(N):                                             
        t = np.linspace(-510, 510, N)
        alpha = np.full(N, 255, dtype=np.uint8)                                              
        colors =  np.round(np.clip(np.stack([-t, 510-np.abs(t), t, alpha], axis=1), 0, 255)).astype(np.uint8)
        return colors / 255.0
    
colors_rgb_pre = spec(12)
print(colors_rgb_pre)
colors_rgb = [colors_rgb_pre[i] for i in train_labels_unsupervised] # list containing rgb colors for each sample in label


fig, ax = plt.subplots()
ax.scatter(principal_df.iloc[:,0], principal_df.iloc[:,1], c=colors_rgb)

legend_labels = [le_name_mapping[i] for i in range(12) if i in le_name_mapping]

legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, markersize=10,
                        markerfacecolor=colors_rgb_pre[i])
                for i, label in le_name_mapping.items()]
ax.legend(handles=legend_handles, labels=legend_labels)
plt.show()

https://stackoverflow.com/questions/50980810/how-to-create-a-discrete-rgb-colourmap-with-n-colours-using-numpy


https://stackoverflow.com/questions/76226587/how-to-add-a-custom-legend-along-with-default-legends

Note: 
Some global structures, will hopefully become clearer when clustering. 
For instance, Torsk, Dypvannsreke, Hyse and Sei seem to be somewhat gathered. 

### KMEANS for PCA 

11 clusters because the dataset of 20,000 samples has 11 species. 

In [ ]:
kmeans_pca = KMeans(n_clusters=11, n_init=15, max_iter=500, random_state=0) 

# Train and make predictions
clusters_pca = kmeans_pca.fit_predict(x_data_unsupervised[:num_unsupervised])

# Cluster centers
centroids = kmeans_pca.cluster_centers_
centroids_pca = pca.transform(centroids)

In [ ]:
clusters_pca[:30] # trenge rikke i modell 1 

In [ ]:
# defining a matrix with the zeros function. Consists of 20 columns (num of fish species) and 11 rows (num of clusters)
arr_cluster = np.zeros([11, 12], dtype = int)
arr_cluster

https://www.geeksforgeeks.org/numpy-zeros-python/

Creating a loop to fill in the zeros array. 
Will create 11 lists (one for each cluster) with the amount of each species in each list. 

In [ ]:
for i in range(num_unsupervised): 
    cluster_number_vertical = clusters_pca[i] # uses the cluster_pca for which number downwards to add 1 
    cluster_number_horisontal = train_labels_unsupervised[i] # uses the train_labels_unsupervised to find out which fish to add to horistonally. 
    arr_cluster[cluster_number_vertical, cluster_number_horisontal] += 1 

In [ ]:
arr_cluster

Using the array to get the percentage of each fish instead. 
Will be used for pie charts.

In [ ]:
# Calculate the percentage of each fish species in each cluster
arr_percentage = arr_cluster/arr_cluster.sum(axis =1)[:, np.newaxis]
print(arr_percentage)

percentages = arr_percentage[0, :] # the first cluster 
labels = label_fishes
threshold = 0.001 

filtered_percentages = [p for p in percentages if p >= threshold] # filters percentage by threshold
filtered_labels = [l for p, l in zip(percentages, labels) if p >= threshold] # filters fish by threshold

#### Pie Chart 1 
Plotting Pie Charts for visualization of the clusters. 

In [ ]:
plt.figure()
plt.pie(filtered_percentages, labels = filtered_labels, autopct = '%1.1f%%')
plt.title("Pie chart")
plt.show()

#### Pie Chart 2

In [ ]:
arr_percentage = arr_cluster/arr_cluster.sum(axis =1)[:, np.newaxis]
percentages = arr_percentage[4, :]
labels = label_fishes
threshold = 0.001

filtered_percentages = [p for p in percentages if p >= threshold] 
filtered_labels = [l for p, l in zip(percentages, labels) if p >= threshold] 

In [ ]:
plt.figure()
plt.pie(filtered_percentages, labels = filtered_labels, autopct = '%1.1f%%')
plt.title("Pie chart")
plt.show()

#### Pie Chart 3 

In [ ]:
arr_percentage = arr_cluster/arr_cluster.sum(axis =1)[:, np.newaxis]
percentages = arr_percentage[8, :]
labels = label_fishes
threshold = 0.001

filtered_percentages = [p for p in percentages if p >= threshold] 
filtered_labels = [l for p, l in zip(percentages, labels) if p >= threshold] 

In [ ]:
plt.figure()
plt.pie(filtered_percentages, labels = filtered_labels, autopct = '%1.1f%%')
plt.title("Pie chart")
plt.show()

Pie chart notes: 

We chose to make three random pie charts from the clustering after KMeans. All of them show one dominant species. 

We see from the first chart (before KMeans) how the top species are divided
All pie charts we created after KMeans show one dominant species. The magnitude of them varies, however, the dominant species in all three are within top 5 most frequently caught. 


### PCA plot 2D 

In [ ]:
plt.figure(figsize=(8,6))

# Scatterplot
plt.scatter(principal_df.iloc[:,0], principal_df.iloc[:,1], c=clusters_pca, cmap="brg", s=40)
plt.scatter(x=centroids_pca[:,0], y=centroids_pca[:,1], marker="x", s=500, linewidths=3, color="black")

# Aesthetics
plt.title('PCA plot in 2D')
plt.xlabel('PC1')
plt.ylabel('PC2')

Notes: 
We see about 9 clearly flocked clusteres. Distinct centers dividing the cluster can signify that KMeans has found sepeartions in the data. 
When comparing the clusters to the plot we see that the bundles from the PCA plot are similar to what we see here. 
There are some outliers and overlapping. This would be interesting to study further. 

In [ ]:
pca_var = PCA()
pca_var.fit(x_data_unsupervised[:num_unsupervised])

# Plot
plt.figure(figsize=(10,5))
xi = np.arange(1, 1+x_data_unsupervised[:num_unsupervised].shape[1], step=1)
yi = np.cumsum(pca_var.explained_variance_ratio_)
plt.plot(xi, yi, marker='o', linestyle='--', color='b')

# Aesthetics
plt.ylim(0.0,1.1)
plt.xlabel('Number of Components')
plt.xticks(np.arange(1, 1+x_data_unsupervised[:num_unsupervised].shape[1], step=1))
plt.ylabel('Cumulative variance (%)')
plt.title('Explained variance by each component')
plt.axhline(y=1, color='r', linestyle='-')
plt.gca().xaxis.grid(False)

Note: 
With only two components the model represents almost 80% of the variance. 
This will increase by adding 1 component (approx. 85%).

## 2. t-SNE

In [ ]:
# t-SNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(x_data_unsupervised[:num_unsupervised])

# Convert to data frame
tsne_df = pd.DataFrame(data = X_tsne, columns = ['tsne comp. 1', 'tsne comp. 2'])


# Shape and preview
print(tsne_df.shape)
tsne_df.head()

We will now apply the same techinque that we used on PCA to compare but without pie charts.

In [ ]:
# def spec generates 20 different colors 
def spec(N):                                             
        t = np.linspace(-510, 510, N)
        alpha = np.full(N, 255, dtype=np.uint8)                                              
        colors =  np.round(np.clip(np.stack([-t, 510-np.abs(t), t, alpha], axis=1), 0, 255)).astype(np.uint8)
        return colors / 255.0
    
colors_rgb_pre = spec(12)
print(colors_rgb_pre)
colors_rgb = [colors_rgb_pre[i] for i in train_labels_unsupervised] # list containing rgb colors for each sample in label


fig, ax = plt.subplots()
ax.scatter(tsne_df.iloc[:,0], tsne_df.iloc[:,1], c=colors_rgb)

legend_labels = [le_name_mapping[i] for i in range(12) if i in le_name_mapping]

legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=label, markersize=10,
                        markerfacecolor=colors_rgb_pre[i])
                for i, label in le_name_mapping.items()]
ax.legend(handles=legend_handles, labels=legend_labels)
plt.show()

In [ ]:
# KMeans
kmeans_tsne = KMeans(n_clusters=11, n_init=15, max_iter=500, random_state=0)

# Train and make predictions
clusters_tsne = kmeans_tsne.fit_predict(x_data_unsupervised[:num_unsupervised])

# Cluster centers
centroids = kmeans_pca.cluster_centers_
centroids_tsne = pca.transform(centroids)

In [ ]:
# Figure size
plt.figure(figsize=(8,6))

# Scatterplot
plt.scatter(tsne_df.iloc[:,0], tsne_df.iloc[:,1], c=clusters_tsne, cmap="brg", s=40)

# Aesthetics
plt.title('t-SNE plot in 2D')
plt.xlabel('tsne component 1')
plt.ylabel('tsne  2')

Notes: 
There are clear clusters here, with what seems to be less overlapping than with the PCA. 
Would be interesting to find out why. 

## 3. UMAP

In [ ]:
# UMAP
um = umap.UMAP()
X_fit = um.fit(x_data_unsupervised[:num_unsupervised])           
X_umap = um.transform(x_data_unsupervised[:num_unsupervised])

# Convert to data frame
umap_df = pd.DataFrame(data = X_umap, columns = ['umap comp. 1', 'umap comp. 2'])

# Shape and preview
print(umap_df.shape)
umap_df.head()

In [ ]:
# UMAP
um = umap.UMAP()
X_fit = um.fit(x_data_unsupervised[:num_unsupervised]) 
X_umap = um.transform(x_data_unsupervised[:num_unsupervised])

# Convert to data frame
umap_df = pd.DataFrame(data = X_umap, columns = ['umap comp. 1', 'umap comp. 2'])

# Shape and preview
print(umap_df.shape)
umap_df.head()

In [ ]:
# Figure size
plt.figure(figsize=(8,6))

# Scatterplot
plt.scatter(umap_df.iloc[:,0], umap_df.iloc[:,1], c=clusters_pca, cmap="brg", s=40)

# Centroids
centroids_umap = um.transform(centroids)
plt.scatter(x=centroids_umap[:,0], y=centroids_umap[:,1], marker="x", s=500, linewidths=3, color="black")

# Aesthetics
plt.title('UMAP plot in 2D')
plt.xlabel('umap component 1')
plt.ylabel('umap component 2')

Notes: 
The initial clustering looks less clustered than the others so we chose to focus on PCA and t-SNE. 

## 4. Conclucion Unsupervised

We think that the clustering methods found some patterns that could be used for understanding the relationships between features and fish species. This will have to be looked further into. With more time, we would find it interesteing to try to use more/less features, especially fishing gear. 
We would like to try to change the amount of clusters, give more/less data, and look at tha charts again, perhaps using the elbow method. The plotting could be enhanced to retrieve a clearer pattern for analysis. Also looking into normalizing the data differently, silhoutte clustering, or other techniques. 
We would especially like to explore t-SNE and UMAP more as these work quite differently than the PCA model. 
